In [478]:
import pandas as pd
import json
import numpy as np
pd.set_option('display.max_colwidth', None)

In [479]:
address = 'kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu'
df = pd.read_csv(f'./data/kujira/{address}.csv')

In [480]:
df['created_at'] = pd.to_datetime(df.created_at)
df['complete_denom'] = df['amount']
df[['amount', 'valuta']] = df['amount'].str.extract(r'(\d+)(.*)')

In [481]:
df['denom'] = df.valuta.replace({
    'factory/kujira1qk00h5atutpsv900x202pxx42npjr9thg58dnqpa72f2p7m2luase444a7/uusk':'uUSK',
    'ibc/295548A78785A1007F232DE286149A6FF512F180AF5657780FC89C009E2C348F': 'axlUSDC',
    'ibc/27394FB092D2ECCD56123C74F36E4C1F926001CEADA9CA97EA622B25F41E5EB2': 'ATOM',
    'ibc/1B38805B1C75352B28169284F96DF56BDEBD9E8FAC005BDCC8CF0378C82AA8E7': 'wETH',
    'ibc/640E1C3E28FD45F611971DF891AE3DC90C825DF759DF8FAA8F33F7F72B35AD56': 'ASTRO',
    'factory/kujira143fwcudwy0exd6zd3xyvqt2kae68ud6n8jqchufu7wdg5sryd4lqtlvvep/urcpt': 'xKUJI',
    'ibc/DA59C009A0B3B95E0549E6BF7B075C8239285989FF457A8EDDBB56F10B2A6986': 'LUNA',
    'ibc/0306D6B66EAA2EDBB7EAD23C0EC9DDFC69BB43E80B398035E90FBCFEF3FD1A87': 'stATOM'
})

In [482]:
with open(f'data/kujira/contracts.json', "r") as json_file:
    contracts = json.load(json_file)

In [483]:
contracts_dict = {}
for contract in contracts:
    contracts_dict[contract['Address']] = contract['Label']

In [484]:
df['contract_label'] = df.contract_address.replace(contracts_dict)

In [485]:
def get_contract_label(row):
    if(row['action'] == '/ibc.core.client.v1.MsgUpdateClient'):
        if(pd.isna(row['contract_label'])):
            return 'IBC Transfer'
    if(row['action'] == '/cosmos.bank.v1beta1.MsgSend'):
        if(pd.isna(row['contract_label'])):
            return 'Transfer'
    return row['contract_label']

In [486]:
df['contract_label'] = df.apply(lambda row: get_contract_label(row), axis=1)

In [487]:
tt = df[((df.address == 'kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu'))]
tt.denom.unique()

array(['ukuji', 'axlUSDC', 'uUSK', 'ATOM', 'wETH', 'ASTRO', 'xKUJI',
       'LUNA', 'stATOM'], dtype=object)

In [488]:
tt[['created_at', 'tx_hash', 'address', 'amount', 'operation', 'action',
       'contract_label', 'contract_action', 'denom']].sort_values(by='created_at')

,created_at,tx_hash,address,amount,operation,action,contract_label,contract_action,denom
1821,2022-07-12 17:44:38+00:00,59FF4DF16DEE76E53404AC5894AD8462FDEC8A2718592AFCAD6B33A99C8BE479,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,49000000,coin_received,/ibc.core.client.v1.MsgUpdateClient,IBC Transfer,NaN,axlUSDC
1816,2022-07-12 17:45:33+00:00,B6AA1AD9BAB1AC75746E31F8B98727F26B63878CAD488F05EC2878F71464A503,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,1090000000,coin_received,/cosmos.bank.v1beta1.MsgSend,Transfer,NaN,ukuji
1807,2022-07-12 17:46:13+00:00,5194346F49923CC1685CD472BCEA43DE49B4251134F06AF76C828552E07DCC8D,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,5000000,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,FIN KUJI-axlUSDC,swap,axlUSDC
1810,2022-07-12 17:46:13+00:00,5194346F49923CC1685CD472BCEA43DE49B4251134F06AF76C828552E07DCC8D,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,12543970,coin_received,/cosmwasm.wasm.v1.MsgExecuteContract,FIN KUJI-axlUSDC,swap,ukuji
1805,2022-07-12 17:46:13+00:00,5194346F49923CC1685CD472BCEA43DE49B4251134F06AF76C828552E07DCC8D,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,4454,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,FIN KUJI-axlUSDC,swap,ukuji
...,...,...,...,...,...,...,...,...,...
89,2023-07-28 18:43:25+00:00,EEFD9E3BE115B7BE86B485857F23877F3C6F536C227799905B861FDA478B0C56,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,9456458,coin_received,/cosmwasm.wasm.v1.MsgExecuteContract,CALC-DCA,swap,ukuji
71,2023-08-04 18:01:21+00:00,1E7AEC0CA968323704290F02B35CA76486FBFD379A19F51F2DD34E312C8D26EA,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,1492658,coin_received,/cosmwasm.wasm.v1.MsgExecuteContract,CALC-DCA,swap,axlUSDC
53,2023-08-04 18:43:23+00:00,3BFABB8565BF7F855073A1F0E31F141253B644CDF87A50D62FAA3A31A5AB1466,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,7080033,coin_received,/cosmwasm.wasm.v1.MsgExecuteContract,CALC-DCA,swap,ukuji
35,2023-08-11 18:01:23+00:00,C2F975BA3CC9E5E6DDD85B61AA0F0A1796CB49DCB4EF0BE2A8B025C431A90792,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,2471288,coin_received,/cosmwasm.wasm.v1.MsgExecuteContract,CALC-DCA,swap,axlUSDC


In [299]:
tt.contract_label.value_counts()

contract_label
Orca Market; USK-ATOM                            97
CALC-DCA                                         95
Orca Market; USK-stATOM                          33
FIN: ATOM-USK                                    30
FIN KUJI-axlUSDC                                 24
USK Market; wETH                                 22
FIN: axlUSDC-USK                                 21
FIN ASTRO-USK                                    10
Enable Orca Market for LUNA collateral on USK     9
Enable LUNA as USK Collateral                     8
FIN ATOM-axlUSDC                                  7
IBC Transfer                                      7
USK Market; ATOM                                  6
FIN: KUJI-USK                                     6
GHOST: Vault: KUJI                                6
FIN stATOM-ATOM                                   6
FIN LUNA-USK                                      3
Transfer                                          2
Name: count, dtype: int64

In [489]:
useful_cols = ['created_at', 'tx_hash', 'address', 'amount', 'operation', 'action', 'contract_label','contract_action', 'denom']

### Association

In [490]:
tt_spent = tt[tt.operation=='coin_spent'][useful_cols]
tt_received = tt[tt.operation=='coin_received'][useful_cols]

#### Spent <- Received

In [491]:
tt_spent_received_merged = tt_spent.merge(tt_received, on=['denom','address'], suffixes=['_spent','_received'])

In [492]:
tt_spent_received_merged['created_at_delta_seconds'] = (tt_spent_received_merged.created_at_spent - tt_spent_received_merged.created_at_received) / np.timedelta64(1, 's')
tt_received_spent_merged_ = tt_spent_received_merged[tt_spent_received_merged['created_at_delta_seconds'] > 0]

In [493]:
tt_spent_received_delta_seconds = tt_received_spent_merged_[['tx_hash_spent','created_at_delta_seconds']].groupby('tx_hash_spent').min().reset_index()

In [494]:
tt_spent_received_merged_delta = tt_received_spent_merged_.merge(tt_spent_received_delta_seconds, on=['tx_hash_spent','created_at_delta_seconds'])

In [495]:
tt_spent_received_merged_delta[tt_spent_received_merged_delta.tx_hash_spent=='2F922451D78A1515ADB0E501358226DE5696DB960322F13696F72C7270D17CEC']

,created_at_spent,tx_hash_spent,address,amount_spent,operation_spent,action_spent,contract_label_spent,contract_action_spent,denom,created_at_received,tx_hash_received,amount_received,operation_received,action_received,contract_label_received,contract_action_received,created_at_delta_seconds
142,2023-03-09 20:59:26+00:00,2F922451D78A1515ADB0E501358226DE5696DB960322F13696F72C7270D17CEC,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,7762496,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,FIN: ATOM-USK,swap,ATOM,2023-03-09 20:58:18+00:00,201746DA98272A1F84398F0EEE7B21226BDF020F9BDEA86E056B90DAB0151678,7762496,coin_received,/cosmwasm.wasm.v1.MsgExecuteContract,Orca Market; USK-ATOM,claim_liquidations,68.0


#### Received <- Spent

In [496]:
tt_received_spent_merged = tt_received.merge(tt_spent, on=['address','contract_label'], suffixes=['_received','_spent'], how='left')

In [497]:
tt_na = tt_received_spent_merged[tt_received_spent_merged.tx_hash_spent.isna()]
tt_received_spent_merged = tt_received_spent_merged[~tt_received_spent_merged.tx_hash_spent.isna()]

In [498]:
tt_received_spent_merged['created_at_delta_seconds'] = (tt_received_spent_merged.created_at_received - tt_received_spent_merged.created_at_spent) / np.timedelta64(1, 's')
tt_received_spent_merged_ = tt_received_spent_merged[tt_received_spent_merged['created_at_delta_seconds'] > 0]

In [499]:
tt_received_spent_delta_seconds = tt_received_spent_merged_[['tx_hash_received','created_at_delta_seconds']].groupby('tx_hash_received').min().reset_index()

In [500]:
tt_received_spent_merged_delta = tt_received_spent_merged_\
                                        .merge(tt_received_spent_delta_seconds, on=['tx_hash_received','created_at_delta_seconds'])

In [501]:
tt_received_spent_merged_delta = pd.concat([tt_received_spent_merged_delta, tt_na])

#### Merge

In [502]:
tt_spent_received_merged_delta['direction'] = 'SPENT'
tt_received_spent_merged_delta['direction'] = 'RECEIVED'
tt_spent_dep = tt_spent_received_merged_delta[['created_at_spent', 'tx_hash_spent', 'address', 'amount_spent',
       'operation_spent', 'action_spent', 'contract_label_spent', 'contract_action_spent', 'denom','direction','tx_hash_received']]
tt_received_dep = tt_received_spent_merged_delta[['created_at_received', 'tx_hash_received', 'address', 'amount_received',
       'operation_received', 'action_received', 'contract_label', 'contract_action_received', 'denom_received','direction','tx_hash_spent']]

In [504]:
tt_spent_dep.columns = ['created_at', 'tx_hash', 'address', 'amount',
       'operation', 'action', 'contract_label', 'contract_action', 'denom','direction','tx_hash_dep']
tt_received_dep.columns = ['created_at', 'tx_hash', 'address', 'amount',
       'operation', 'action', 'contract_label', 'contract_action', 'denom','direction','tx_hash_dep']
tt_deps = pd.concat([tt_spent_dep, tt_received_dep])

### Export

In [505]:
hashes = set(['58CB6764F9BFE2DDD1ED9794DCB864E3E6B810682C68C46F3446EA30E8907670'])
while True:
    tt_ = tt_deps[tt_deps.tx_hash.isin(hashes)]
    _hashes = hashes.union(set(tt_.tx_hash.values)).union(set(tt_.tx_hash_dep.values))
    if len(_hashes) == len(hashes):
        break
    hashes = hashes.union(_hashes)

In [506]:
tt_

,created_at,tx_hash,address,amount,operation,action,contract_label,contract_action,denom,direction,tx_hash_dep
1,2023-07-03 13:29:44+00:00,6DD89FC7A77963604B95471EBB893B1A5E8CFD65D818D24CA1A4C17E60D5B558,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,2722,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,Orca Market; USK-ATOM,claim_liquidations,ukuji,SPENT,F6B527BD1454BA41F0F27A7D6D49A8607870E5264535F3C8D10C28139CCAF746
2,2023-06-26 15:13:29+00:00,F5A8867D36868E4691CC8EEE30398FE2F01531EFC4BFF164520727F11EF84AAF,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,2268,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,Orca Market; USK-ATOM,activate_bids,ukuji,SPENT,2E565E00AA69AECAFAE88134F9386FC3F209A250406E22DBB78B5A7D87DFDA7C
5,2023-06-14 21:03:36+00:00,5B252B19DCE3707EDC72462FC8CFADE1AD12BFCD22FBE58A907A5CC7801989DC,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,3432,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,FIN: ATOM-USK,retract_orders,ukuji,SPENT,7B17071AC208501B367374D95B7C888C921249ADFDE52BC000121E629407A64B
6,2023-06-10 16:05:47+00:00,7CE2F5BD0497C37EEB13F3844601AF5F6BC7DB48151EABA805A9AF0BBE9B9DBD,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,2428,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,Orca Market; USK-ATOM,retract_bid,ukuji,SPENT,7B17071AC208501B367374D95B7C888C921249ADFDE52BC000121E629407A64B
7,2023-06-10 07:55:39+00:00,F71C334D04493F5137987580FC6D85BE4D01D9467A56E6F4B3B7A014695870BB,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,2249,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,Orca Market; USK-ATOM,activate_bids,ukuji,SPENT,7B17071AC208501B367374D95B7C888C921249ADFDE52BC000121E629407A64B
14,2023-06-10 04:18:26+00:00,110D733773661BB5CAAF93E683EB466620DD56F25093D5FBEBFC251939CF767E,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,2327,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,FIN: ATOM-USK,retract_orders,ukuji,SPENT,7B17071AC208501B367374D95B7C888C921249ADFDE52BC000121E629407A64B
20,2023-06-09 21:48:19+00:00,3C1D9C29A813A402761B2A8F95FFFAB1DF801E29ACB21919608E7D22DA93E703,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,2721,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,Orca Market; USK-ATOM,claim_liquidations,ukuji,SPENT,7B17071AC208501B367374D95B7C888C921249ADFDE52BC000121E629407A64B
21,2023-06-09 08:40:19+00:00,E49B63B5F76D33EF2F59313429924495EFEE50AAAC55328F5AE854B0DB340ADC,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,2268,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,Orca Market; USK-ATOM,activate_bids,ukuji,SPENT,15A5F10B69A59B427EE6A2DB7D2F6F8FB9E0837B8CE34B3BE9355D024101B770
22,2023-06-09 08:40:19+00:00,E49B63B5F76D33EF2F59313429924495EFEE50AAAC55328F5AE854B0DB340ADC,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,2268,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,Orca Market; USK-ATOM,activate_bids,ukuji,SPENT,15A5F10B69A59B427EE6A2DB7D2F6F8FB9E0837B8CE34B3BE9355D024101B770
31,2023-06-02 18:39:07+00:00,9BD5C897D2188782378D28643BE33B0DB2DA6B3385A06CEC1A23DEE83CF6A462,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,495,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,CALC-DCA,NaN,ukuji,SPENT,15A5F10B69A59B427EE6A2DB7D2F6F8FB9E0837B8CE34B3BE9355D024101B770


In [473]:
df_to_export = tt_

In [477]:
df_to_export.sort_values(by='created_at')

,created_at,tx_hash,address,amount,operation,action,contract_label,denom,direction,tx_hash_dep
2665,2023-04-21 20:43:55+00:00,ECC45D8609833BD1670596B529C011761E047E26F8092BEC39E1851DDC95A992,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,49800000,coin_received,/ibc.core.client.v1.MsgUpdateClient,IBC Transfer,axlUSDC,RECEIVED,NaN
150,2023-04-21 20:46:00+00:00,3721DDA844D609D62F8A4AD3BE50880D2DD2B240DF0E5BEC1CB6536C1E1FAFB4,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,30000000,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,CALC-DCA,axlUSDC,SPENT,ECC45D8609833BD1670596B529C011761E047E26F8092BEC39E1851DDC95A992
101,2023-05-22 12:22:35+00:00,3C03BFF8FEE6754021BDF7FE393B893916A830C7D4FA6A4519FAE08E48AEA843,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,6579521,coin_received,/cosmwasm.wasm.v1.MsgExecuteContract,CALC-DCA,ukuji,RECEIVED,3721DDA844D609D62F8A4AD3BE50880D2DD2B240DF0E5BEC1CB6536C1E1FAFB4
102,2023-05-22 12:22:35+00:00,3C03BFF8FEE6754021BDF7FE393B893916A830C7D4FA6A4519FAE08E48AEA843,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,6579521,coin_received,/cosmwasm.wasm.v1.MsgExecuteContract,CALC-DCA,ukuji,RECEIVED,3721DDA844D609D62F8A4AD3BE50880D2DD2B240DF0E5BEC1CB6536C1E1FAFB4
53,2023-05-25 18:17:36+00:00,1874B02F78A7D7BF88010AE3E8C922B2BFB67B7C10F360D4F0A214C2A9AFF599,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,90059790,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,FIN: KUJI-USK,ukuji,SPENT,3C03BFF8FEE6754021BDF7FE393B893916A830C7D4FA6A4519FAE08E48AEA843
52,2023-05-25 18:17:36+00:00,1874B02F78A7D7BF88010AE3E8C922B2BFB67B7C10F360D4F0A214C2A9AFF599,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,353,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,FIN: KUJI-USK,ukuji,SPENT,3C03BFF8FEE6754021BDF7FE393B893916A830C7D4FA6A4519FAE08E48AEA843
97,2023-05-27 16:33:30+00:00,2EBBC7461422BAC9EFCDF01DF7EF142B366D803E189725A861C2187D1D0A6652,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,150099767,coin_received,/cosmwasm.wasm.v1.MsgExecuteContract,FIN: KUJI-USK,ukuji,RECEIVED,1874B02F78A7D7BF88010AE3E8C922B2BFB67B7C10F360D4F0A214C2A9AFF599
98,2023-05-27 16:33:30+00:00,2EBBC7461422BAC9EFCDF01DF7EF142B366D803E189725A861C2187D1D0A6652,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,150099767,coin_received,/cosmwasm.wasm.v1.MsgExecuteContract,FIN: KUJI-USK,ukuji,RECEIVED,1874B02F78A7D7BF88010AE3E8C922B2BFB67B7C10F360D4F0A214C2A9AFF599
51,2023-05-27 16:33:30+00:00,2EBBC7461422BAC9EFCDF01DF7EF142B366D803E189725A861C2187D1D0A6652,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,399,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,FIN: KUJI-USK,ukuji,SPENT,3C03BFF8FEE6754021BDF7FE393B893916A830C7D4FA6A4519FAE08E48AEA843
46,2023-05-27 16:35:41+00:00,BB22108711B3977B7F158B369007077B76ADD4179DCF2B650E3C162F6B28FF67,kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu,33771467,coin_spent,/cosmwasm.wasm.v1.MsgExecuteContract,FIN KUJI-axlUSDC,ukuji,SPENT,2EBBC7461422BAC9EFCDF01DF7EF142B366D803E189725A861C2187D1D0A6652


In [475]:
df1 = df_to_export[['tx_hash']]
df1.columns = ['address']
df2 = df_to_export[['tx_hash_dep']]
df2.columns = ['address']
pd.concat([df1, df2])\
        .drop_duplicates(ignore_index=True)\
        .to_csv(f'data/kujira/graph/nodes.csv', index=False)

In [476]:
df_to_export.to_csv(f'data/kujira/graph/links.csv', index=False)